# DeepSeek R1 UD-IQ2_XXS llama.cpp serving using the Azure ML Python SDK

> [1] Please use `Python 3.10 - SDK v2 (azureml_py310_sdkv2)` conda environment.<br>[2] Please make sure you prepare [Hugging Face API Token](https://huggingface.co/docs/hub/security-tokens).

## Download Preprocessed Quantized Models

You may first install `huggingface-cli` to download huggingface models.

In [ ]:
%pip install -U "huggingface_hub[cli]"

In [1]:
!huggingface-cli login --token <Your Token>

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /home/azureuser/.cache/huggingface/token
Login successful


In [ ]:
!huggingface-cli download unsloth/DeepSeek-R1-GGUF --quiet --include "*UD-IQ2_XXS*" --local-dir DeepSeek-R1-GGUF --cache-dir .cache

## 1. Load config file

---


In [ ]:
import os
import yaml
from logger import logger
from datetime import datetime

snapshot_date = datetime.now().strftime("%Y-%m-%d")

with open("config.yml") as f:
    d = yaml.load(f, Loader=yaml.FullLoader)

AZURE_SUBSCRIPTION_ID = d["config"]["AZURE_SUBSCRIPTION_ID"]
AZURE_RESOURCE_GROUP = d["config"]["AZURE_RESOURCE_GROUP"]
AZURE_WORKSPACE = d["config"]["AZURE_WORKSPACE"]
HF_TOKEN = ''
HF_MODEL_NAME_OR_PATH = d["config"]["HF_MODEL_NAME_OR_PATH"]
IS_DEBUG = d["config"]["IS_DEBUG"]

azure_env_name = 'deepseek-llm-cpp-r1-q2'
azure_model_name = 'deepseek-r1-q2'
azure_endpoint_name = 'deepseek-r1-q2-endpoint'
azure_deployment_name = 'blue'
azure_serving_cluster_size = 'Standard_ND40rs_v2'


if IS_DEBUG:
    logger.debug("===== 0. Azure ML Deployment Info =====")
    logger.debug(f"AZURE_SUBSCRIPTION_ID={AZURE_SUBSCRIPTION_ID}")
    logger.debug(f"AZURE_RESOURCE_GROUP={AZURE_RESOURCE_GROUP}")
    logger.debug(f"AZURE_WORKSPACE={AZURE_WORKSPACE}")
    logger.debug(f"HF_MODEL_NAME_OR_PATH={HF_MODEL_NAME_OR_PATH}")
    logger.debug(f"IS_DEBUG={IS_DEBUG}")

    logger.debug(f"azure_env_name={azure_env_name}")
    logger.debug(f"azure_model_name={azure_model_name}")
    logger.debug(f"azure_endpoint_name={azure_endpoint_name}")
    logger.debug(f"azure_deployment_name={azure_deployment_name}")
    logger.debug(f"azure_serving_cluster_size={azure_serving_cluster_size}")

2025-03-05 13:51:23,805 - logger - DEBUG - ===== 0. Azure ML Deployment Info =====
2025-03-05 13:51:23,805 - logger - DEBUG - AZURE_SUBSCRIPTION_ID=<your-subscription-id>
2025-03-05 13:51:23,805 - logger - DEBUG - AZURE_RESOURCE_GROUP=<your-resource-group>
2025-03-05 13:51:23,805 - logger - DEBUG - AZURE_WORKSPACE=<your-workspace-name>
2025-03-05 13:51:23,812 - logger - DEBUG - HF_MODEL_NAME_OR_PATH=DeepSeek-R1-GGUF/DeepSeek-R1-UD-IQ2_XXS
2025-03-05 13:51:23,812 - logger - DEBUG - IS_DEBUG=True
2025-03-05 13:51:23,812 - logger - DEBUG - azure_env_name=deepseek-llm-cpp-r1-q2
2025-03-05 13:51:23,812 - logger - DEBUG - azure_model_name=deepseek-r1-q2
2025-03-05 13:51:23,812 - logger - DEBUG - azure_endpoint_name=deepseek-r1-q2-endpoint
2025-03-05 13:51:23,812 - logger - DEBUG - azure_deployment_name=blue
2025-03-05 13:51:23,812 - logger - DEBUG - azure_serving_cluster_size=Standard_ND40rs_v2


<br>

## 2. Serving preparation

---

### 2.1. Configure workspace details

To connect to a workspace, we need identifying parameters - a subscription, a resource group, and a workspace name. We will use these details in the MLClient from azure.ai.ml to get a handle on the Azure Machine Learning workspace we need. We will use the default Azure authentication for this hands-on.


In [ ]:
# %pip install -q azure-ai-ml azure-identity

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
azureml-mlflow 1.57.0.post1 requires azure-storage-blob<=12.19.0,>=12.5.0, but you have azure-storage-blob 12.24.1 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [4]:
# import required libraries
import time
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient, Input
from azure.ai.ml import command
from azure.ai.ml.entities import Model
from azure.ai.ml.constants import AssetTypes
from azure.core.exceptions import ResourceNotFoundError, ResourceExistsError

logger.info(f"===== 2. Serving preparation =====")
logger.info(f"Calling DefaultAzureCredential.")
credential = DefaultAzureCredential()
ml_client = None
try:
    ml_client = MLClient.from_config(credential)
except Exception as ex:
    print(ex)
    ml_client = MLClient(
        credential, AZURE_SUBSCRIPTION_ID, AZURE_RESOURCE_GROUP, AZURE_WORKSPACE
    )

2025-03-04 01:05:20,030 - logger - INFO - ===== 2. Serving preparation =====
2025-03-04 01:05:20,031 - logger - INFO - Calling DefaultAzureCredential.
Found the config file in: /config.json


### 2.2. Create model asset


In [8]:
def get_or_create_model_asset(
    ml_client,
    model_name,
    job_name=None,
    model_dir="outputs",
    model_type="custom_model",
    update=False,
):
    try:
        latest_model_version = max(
            [int(m.version) for m in ml_client.models.list(name=model_name)]
        )
        if update:
            raise ResourceExistsError("Found Model asset, but will update the Model.")
        else:
            model_asset = ml_client.models.get(
                name=model_name, version=latest_model_version
            )
            logger.info(f"Found Model asset: {model_name}. Will not create again")
    except (ResourceNotFoundError, ResourceExistsError, ValueError) as e:
        logger.info(f"Exception: {e}")
        run_model = Model(
            name=model_name,
            path=model_dir,
            description=f"Deepseek model {HF_MODEL_NAME_OR_PATH}",
            type=model_type,  # mlflow_model, custom_model, triton_model
        )
        model_asset = ml_client.models.create_or_update(run_model)
        logger.info(f"Created Model asset: {model_name} from {model_dir}")

    return model_asset

In [9]:
model = get_or_create_model_asset(
    ml_client,
    azure_model_name,
    job_name=None,
    model_dir="DeepSeek-R1-GGUF/DeepSeek-R1-UD-IQ2_XXS",
    model_type="custom_model",
    update=False,
)

2025-03-04 01:08:18,627 - logger - INFO - Exception: (UserError) The specified resource was not found.
Code: UserError
Message: The specified resource was not found.
Exception Details:	(ModelNotFound) Model container with name: deepseek-r1-q2 not found.
	Code: ModelNotFound
	Message: Model container with name: deepseek-r1-q2 not found.
Your file exceeds 100 MB. If you experience low speeds, latency, or broken connections, we recommend using the AzCopyv10 tool for this file transfer.

Example: azcopy copy '/mnt/batch/tasks/shared/LS_root/mounts/clusters/jihualiu3/code/llm-inferencing/DeepSeek-R1-GGUF/DeepSeek-R1-UD-IQ2_XXS' 'https://testvllm3226465244.blob.core.windows.net/azureml-blobstore-10331bed-c16d-4680-834e-e054c4eeaeb9/LocalUpload/58609cbc8097dc0357f15b6f8a28407d/DeepSeek-R1-UD-IQ2_XXS' 

See https://learn.microsoft.com/azure/storage/common/storage-use-azcopy-v10 for more information.
Uploading DeepSeek-R1-UD-IQ2_XXS (196162.48 MBs): 100%|██████████| 196162482747/196162482747 [1

### 2.3. Create AzureML environment

Azure ML defines containers (called environment asset) in which your code will run. We can use the built-in environment or build a custom environment (Docker container, conda). This hands-on uses Docker container.


#### Docker environment


In [10]:
from azure.ai.ml.entities import Environment, BuildContext


def get_or_create_docker_environment_asset(
    ml_client, env_name, docker_dir, inference_config=None, update=False
):

    try:
        latest_env_version = max(
            [int(e.version) for e in ml_client.environments.list(name=env_name)]
        )
        if update:
            raise ResourceExistsError(
                "Found Environment asset, but will update the Environment."
            )
        else:
            env_asset = ml_client.environments.get(
                name=env_name, version=latest_env_version
            )
            logger.info(f"Found Environment asset: {env_name}. Will not create again")
    except (ResourceNotFoundError, ResourceExistsError) as e:
        logger.info(f"Exception: {e}")
        env_docker_image = Environment(
            build=BuildContext(path=docker_dir),
            name=env_name,
            description="Environment created from a Docker context.",
            inference_config=inference_config,
        )
        env_asset = ml_client.environments.create_or_update(env_docker_image)
        logger.info(f"Created Environment asset: {env_name}")

    return env_asset


inference_config = {
    "liveness_route": {
        "port": 8000,
        "path": "/health",
    },
    "readiness_route": {
        "port": 8000,
        "path": "/health",
    },
    "scoring_route": {
        "port": 8000,
        "path": "/",
    },
}

env = get_or_create_docker_environment_asset(
    ml_client, azure_env_name, "docker-r1", inference_config, update=True
)

2025-03-04 03:16:39,944 - logger - INFO - Exception: (UserError) System.Net.Http.HttpConnectionResponseContent
Code: UserError
Message: System.Net.Http.HttpConnectionResponseContent
2025-03-04 03:16:42,664 - logger - INFO - Created Environment asset: deepseek-llm-cpp-r1-q2


<br>

## 3. Serving

---

### 3.1. Create endpoint

Create an endpoint. This process does not provision a GPU cluster yet.


In [ ]:
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
)

logger.info(f"===== 3. Serving =====")

t0 = time.time()

# Check if the endpoint already exists in the workspace
try:
    endpoint = ml_client.online_endpoints.get(azure_endpoint_name)
    logger.info("---Endpoint already exists---")
except:
    # Create an online endpoint if it doesn't exist

    # Define the endpoint
    endpoint = ManagedOnlineEndpoint(
        name=azure_endpoint_name,
        description=f"Test endpoint for {model.name}",
    )

    # Trigger the endpoint creation
    try:
        ml_client.begin_create_or_update(endpoint).wait()
        logger.info("\n---Endpoint created successfully---\n")
    except Exception as err:
        raise RuntimeError(f"Endpoint creation failed. Detailed Response:\n{err}") from err

t1 = time.time()

from humanfriendly import format_timespan

timespan = format_timespan(t1 - t0)
logger.info(f"Creating Endpoint took {timespan}")

2025-03-04 03:16:54,432 - logger - INFO - ===== 3. Serving =====
2025-03-04 03:17:57,612 - logger - INFO - 
---Endpoint created successfully---

2025-03-04 03:17:57,665 - logger - INFO - Creating Endpoint took 1 minute and 3.21 seconds


### 3.2. Create Deployment

Create a Deployment. This takes a lot of time as GPU clusters must be provisioned and the serving environment must be built.


In [12]:
env_vars = {
    "MODEL_NAME": '/models/DeepSeek-R1-UD-IQ2_XXS/DeepSeek-R1-UD-IQ2_XXS-00001-of-00004.gguf', ## /var/azureml-app/azureml-models/deepseek-adapter/DeepSeek-R1-Distill-Qwen-1.5B
    "LAYER_N": "61",
    "PREDICT_N": "10000",
    "VLLM_ARGS": "",
}
deployment_env_vars = {**env_vars}

In [ ]:
%%time
import time
from azure.ai.ml.entities import (    
    OnlineRequestSettings,
    CodeConfiguration,
    ManagedOnlineDeployment,
    ProbeSettings,
    Environment
)

t0 = time.time()
deployment = ManagedOnlineDeployment(
    name=azure_deployment_name,
    endpoint_name=azure_endpoint_name,
    model=model,
    model_mount_path='/models',
    instance_type=azure_serving_cluster_size,
    instance_count=1,
    environment_variables=deployment_env_vars,    
    environment=env,
    request_settings=OnlineRequestSettings(
        max_concurrent_requests_per_instance=2,
        request_timeout_ms=120000, 
        max_queue_wait_ms=240000
    ),
    liveness_probe=ProbeSettings(
        failure_threshold=5,
        success_threshold=1,
        timeout=10,
        period=30,
        initial_delay=120
    ),
    readiness_probe=ProbeSettings(
        failure_threshold=30,
        success_threshold=1,
        timeout=2,
        period=10,
        initial_delay=120,
    ),
)

# Trigger the deployment creation
try:
    ml_client.begin_create_or_update(deployment).wait()
    logger.info("\n---Deployment created successfully---\n")
except Exception as err:
    raise RuntimeError(
        f"Deployment creation failed. Detailed Response:\n{err}"
    ) from err
    
endpoint.traffic = {azure_deployment_name: 100}
endpoint_poller = ml_client.online_endpoints.begin_create_or_update(endpoint)

t1 = time.time()
timespan = format_timespan(t1 - t0)
logger.info(f"Creating deployment took {timespan}")

Check: endpoint deepseek-r1-q2-endpoint exists


.....................................................................................................................................................................

2025-03-04 03:32:10,707 - logger - INFO - 
---Deployment created successfully---

2025-03-04 03:32:12,449 - logger - INFO - Creating deployment took 14 minutes and 14.76 seconds


CPU times: user 2.06 s, sys: 180 ms, total: 2.24 s
Wall time: 14min 14s


In [14]:
endpoint_results = endpoint_poller.result()

In [15]:
print(endpoint_results)

auth_mode: key
description: Test endpoint for deepseek-r1-q2
id: /subscriptions/e56790f8-0506-49eb-95b8-82817828d59d/resourceGroups/dev/providers/Microsoft.MachineLearningServices/workspaces/test-vllm/onlineEndpoints/deepseek-r1-q2-endpoint
identity:
  principal_id: 2a38e9ed-6182-4ade-97a3-26d5ef1ddb9b
  tenant_id: 16b3c013-d300-468d-ac64-7eda0820b6d3
  type: system_assigned
kind: Managed
location: westeurope
mirror_traffic: {}
name: deepseek-r1-q2-endpoint
openapi_uri: https://deepseek-r1-q2-endpoint.westeurope.inference.ml.azure.com/swagger.json
properties:
  AzureAsyncOperationUri: https://management.azure.com/subscriptions/e56790f8-0506-49eb-95b8-82817828d59d/providers/Microsoft.MachineLearningServices/locations/westeurope/mfeOperationsStatus/oeidp:10331bed-c16d-4680-834e-e054c4eeaeb9:40d67bd9-3824-47cf-a0a6-8f5e5b1ed5f2?api-version=2022-02-01-preview
  azureml.onlineendpointid: /subscriptions/e56790f8-0506-49eb-95b8-82817828d59d/resourcegroups/dev/providers/microsoft.machinelearni

In [16]:
endpoint_name = endpoint_results.name
keys = ml_client.online_endpoints.get_keys(name=endpoint_name)
primary_key = keys.primary_key

<br>

## 4. Test

---

### 4.1. Invocation

Try calling the endpoint.


In [40]:
%pip install -q openai

Note: you may need to restart the kernel to use updated packages.


In [17]:
from openai import OpenAI

url = os.path.join(endpoint_results.scoring_uri, "v1")
endpoint_name = (
    endpoint_results.name if azure_endpoint_name is None else azure_endpoint_name
)
keys = ml_client.online_endpoints.get_keys(name=endpoint_name)
primary_key = keys.primary_key  # You can paste [YOUR Azure ML API KEY] here
llm = OpenAI(base_url=url, api_key=primary_key)
model_path = "/models/DeepSeek-R1-UD-IQ2_XXS"

In [18]:
# Create your prompt
system_message = """
You are an AI assistant that helps customers find information. As an assistant, you respond to questions in a concise and unique manner.
You can use Markdown to answer simply and concisely, and add a personal touch with appropriate emojis.

Add a witty joke starting with "By the way," at the end of your response. Do not mention the customer's name in the joke part.
The joke should be related to the specific question asked.
For example, if the question is about tents, the joke should be specifically related to tents.

Use the given context to provide a more personalized response. Write each sentence on a new line:
"""
context = """
    The Alpine Explorer Tent features a detachable partition to ensure privacy, 
    numerous mesh windows and adjustable vents for ventilation, and a waterproof design. 
    It also includes a built-in gear loft for storing outdoor essentials. 
    In short, it offers a harmonious blend of privacy, comfort, and convenience, making it a second home in nature!
"""
question = "What are features of the Alpine Explorer Tent?"

user_message = f"""
Context: {context}
Question: {question}
"""

Simple API Call

In [19]:
# Simple API Call
response = llm.chat.completions.create(
    model=model_path,
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message},
    ],
    temperature=0.7,
    max_tokens=200,
)

print(response.choices[0].message.content)

<think>
Alright, let's tackle this question about the Alpine Explorer Tent. The user is asking for its features, so I need to go through the context provided.

First, the context mentions a detachable partition for privacy. That's a key point. Then there are mesh windows and adjustable vents for ventilation. The waterproof design is another important feature. Also, the built-in gear loft for storage. The summary says it's a blend of privacy, comfort, and convenience, like a second home. 

I need to list these features clearly. Each sentence on a new line, using markdown. Then add a joke related to tents. The joke should start with "By the way," and not include the user's name. Let me think of a tent joke. Maybe something about camping and not needing a room, since the tent has privacy. "Why did the tent refuse to gossip? It had too many *vents* to cover!" Yeah, that works with vents and privacy. Al


Streaming

In [20]:
response = llm.chat.completions.create(
    model=model_path,
    messages=[
        {"role": "saystem", "content": system_message},
        {"role": "user", "content": user_message},
    ],
    temperature=0.7,
    max_tokens=200,
    stream=True,  # Stream the response
)

print("Streaming response:")
for chunk in response:
    delta = chunk.choices[0].delta
    if hasattr(delta, "content"):
        print(delta.content, end="", flush=True)

Streaming response:
<think>
Okay, let's see. The user is asking about the features of the Alpine Explorer Tent. Let me go through the context provided again to make sure I don't miss anything.

The context mentions a detachable partition for privacy. That's one feature. Then there are numerous mesh windows and adjustable vents, which are for ventilation. So ventilation features include those two points. The tent is waterproof, which is a key design aspect. Also, there's a built-in gear loft for storing outdoor essentials. 

The summary part says it blends privacy, comfort, and convenience, acting as a second home. I need to list all the specific features from the context. Let me check again to ensure I don't include the summary as a feature itself but extract the actual components. Detachable partition, mesh windows, adjustable vents, waterproof design, built-in gear loft. Those are the explicit features. The rest is more about the benefits or the overall impression. 

I should present

Another method

In [21]:
import requests

completions_url = os.path.join(endpoint_results.scoring_uri, "v1/completions")
headers = {"Content-Type": "application/json", "Authorization": f"Bearer {primary_key}"}
data = {
    "model": model_path,
    "prompt": "San Francisco is a ",
    "max_tokens": 200,
    "temperature": 0.7,
}

response = requests.post(completions_url, headers=headers, json=data)
print(response.json())

{'choices': [{'text': '7.5-square-mile city where 850,000 people live, work, and play. As the population grows and climate change impacts become more severe, the need for resilient infrastructure and public spaces is critical. San Francisco has been a leader in green infrastructure, with a long history of progressive environmental policies. San Francisco was one of the first cities in the nation to develop a Green Connections plan, which is a citywide network of green, social, and bicycle and pedestrian corridors connecting people to parks, open spaces, and the waterfront. The plan is a result of an extensive community engagement process, and it serves as the city’s green infrastructure plan. San Francisco has also been a leader in stormwater management, with a long history of using green infrastructure to manage stormwater. The city’s Stormwater Management Plan was adopted in 2006, and it includes a comprehensive approach to stormwater management that includes both traditional gray in

### 4.2. LLM latency/throughput simple benchmarking


In [22]:
import numpy as np
from time import perf_counter


def simple_llm_benchmark(
    llm: OpenAI,
    messages: list,
    model_path: str = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B",
    num_warmups: int = 1,
    num_infers: int = 5,
    **params: dict,
) -> dict:

    print("=== Measuring latency ===")
    print(f"model_path={model_path}, num_infers={num_infers}, params={params}")

    latencies = []
    # Warm up
    for _ in range(num_warmups):
        response = llm.chat.completions.create(
            model=model_path,
            messages=messages,
            **params,
        )
    print("=== Warmup done. Start Benchmarking... ===")
    begin = time.time()
    # Timed run
    for curr_infer in range(num_infers):
        start_time = perf_counter()
        if (curr_infer % 5) == 0:
            print(f"Inferring {curr_infer}th...")
        response = llm.chat.completions.create(
            model=model_path,
            messages=messages,
            **params,
        )
        latency = perf_counter() - start_time
        latencies.append(latency)
    end = time.time()

    # Compute run statistics
    duration = end - begin
    time_avg_sec = np.mean(latencies)
    time_std_sec = np.std(latencies)
    time_p95_sec = np.percentile(latencies, 95)
    time_p99_sec = np.percentile(latencies, 99)

    # Metrics
    metrics = {
        "duration": duration,
        "avg_sec": time_avg_sec,
        "std_sec": time_std_sec,
        "p95_sec": time_p95_sec,
        "p99_sec": time_p99_sec,
    }

    return metrics

In [23]:
messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_message},
]
params = {
    "max_tokens": 100,
    "temperature": 0.5,
}

model_path = "/models/DeepSeek-R1-UD-IQ2_XXS"

metrics = simple_llm_benchmark(
    llm, messages, model_path=model_path, num_warmups=1, num_infers=10, **params
)

=== Measuring latency ===
model_path=/models/DeepSeek-R1-UD-IQ2_XXS, num_infers=10, params={'max_tokens': 100, 'temperature': 0.5}
=== Warmup done. Start Benchmarking... ===
Inferring 0th...
Inferring 5th...


In [25]:
import pprint

pprint.pprint(metrics)

{'avg_sec': 7.865846022090409,
 'duration': 78.65847659111023,
 'p95_sec': 8.272176076984033,
 'p99_sec': 8.30625159942545,
 'std_sec': 0.3825325533353871}


## Clean up


In [26]:
ml_client.online_endpoints.begin_delete(azure_endpoint_name)

......................................................................................